# Step 1
We import the .csv file of the Database of Art Historians filtered by nationality ('it') and manually integrated with Wikidata info. Then, we create a pandas dataframe to easily store and manipulate this information.

In [8]:
import pandas as pd

# python3 -m pip install qwikidata
# python library for working with sparql and linked data from WikiData
from qwikidata.sparql import return_sparql_query_results

In [9]:
# create first dataframe only using the specified columns 
data = pd.read_csv("DoAH_StoriciItaliani_integrato.csv", sep=";",
                    usecols=["Full Name", "Gender", "Collection", "Keeper"])

# axis 0 to drop the rows, subset to only remove NaNs from the column Archives
data.dropna(axis=0, subset=["Keeper"], inplace=True)

# resetting the index because all deleted rows have changed the length of the dataframe
data.reset_index(inplace=True, drop=True)

# .pickle is a python serialization format for easy and quick read-write, and pandas supports it natively
data.to_pickle("00_first_db.pickle")

# the first table we have looks like this:
data.head()

,Full Name,Gender,Collection,Keeper
0,"Accascina, Maria",female,NaN,Comune di Palermo\n
1,"Agostini, Leonardo",male,NaN,Scuola Normale Superiore
2,"Alfieri, Vittorio",male,NaN,Biblioteca Medicea Laurenziana
3,"Alinari, Giuseppe",male,Archivio Alinari,Museo Nazionale Alinari della Fotografia
4,"Alinari, Leopoldo",male,Archivio Alinari,Museo Nazionale Alinari della Fotografia


# To resolve:
1. full names are reversed (`surname, name`)
2. we need to have a controlled entity (`wd:xyz`) for each name and keeper, to be able to link them to other info

In [7]:
def reformat_names(name):
    """ reverse names from surname,name format to name surname """
    l = name.split(", ")
    new = " ".join(reversed(l))
    return new

In [12]:
# reverse names and remove duplicate whitespace
data["Full Name"] = data["Full Name"].apply(reformat_names)

data.describe()

,Full Name,Gender,Collection,Keeper
count,119,119,50,119
unique,119,2,49,79
top,Maria Accascina,male,Archivio Alinari,BEIC Digital Library
freq,1,108,2,10


In [13]:
historian_entity_from_label = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?artHistorian

WHERE {{
    OPTIONAL{{
        ?artHistorian wdt:P106 wd:Q1792450 .
    }}
    ?artHistorian wdt:P31 wd:Q5 .
    ?artHistorian rdfs:label ?o 
                  FILTER ( str(?o) = "{}" )  .
}}
"""

In [14]:
def find_historian_entity_from_name(name: str):
    query = historian_entity_from_label.format(name)
    res = return_sparql_query_results(query_string=query)
    try:
        wdt_uri = res['results']['bindings'][0]['artHistorian']['value']
    except (IndexError, KeyError):
        return ""
    return wdt_uri.split("/")[-1]

In [15]:
data["Historian Entity"] = data["Full Name"].apply(find_historian_entity_from_name)

,Full Name,Gender,Collection,Keeper,Historian Entity
0,Maria Accascina,female,NaN,Comune di Palermo\n,Q98804253
1,Leonardo Agostini,male,NaN,Scuola Normale Superiore,
2,Vittorio Alfieri,male,NaN,Biblioteca Medicea Laurenziana,
3,Giuseppe Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,
4,Leopoldo Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,
...,...,...,...,...,...
95,Giovanni Previtali,male,Fondo Giovanni Previtali,University of Siena,Q3107317
96,Ugo Procacci,male,NaN,Camaiore Religious Art Museum,Q2474041
97,Isa Ragusa,female,NaN,Scuola d'Italia Guglielmo Marconi,
98,Corrado Ricci,male,Fondo Corrado Ricci,Istituto Nazionale di Archeologia e Storia del...,Q3694305


In [16]:
data.to_pickle("00_first_db.pickle")

In [17]:
pd.set_option("display.max_rows", None)

In [61]:
historian_entity_from_label_edited = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?artHistorian

WHERE {{
    OPTIONAL {{
      ?artHistorian wdt:P1412 wd:Q652
    }}
  
    ?artHistorian rdfs:label ?o 
                  FILTER ( str(?o) = "{}" )  .
    
  
    ?artHistorian wdt:P31 wd:Q5 .
}}
"""

In [62]:
def find_remaining_entity_from_name(row):
    if row["Historian Entity"]:
        return row["Historian Entity"]
    name = row["Full Name"]
    query = historian_entity_from_label_edited.format(name)
    res = return_sparql_query_results(query_string=query)
    try:
        wdt_uri = res['results']['bindings'][0]['artHistorian']['value']
    except (IndexError, KeyError):
        return ""
    return wdt_uri.split("/")[-1]

In [63]:
data = pd.read_pickle("00_first_db.pickle")

data["Historian Entity"] = data[["Full Name", "Historian Entity"]].apply(find_remaining_entity_from_name, axis=1)


Q98804253
Q1121086
Q778445
Q979574
Q19754060
Q1058859
Q19753744
Q471179
Q18934852
Q572091
Q428638
Q3617742
Q3892735
Q3768943
Q1010831
Q3055532
Q708563
Q2754987
Q913963
Q721545
Q192348
Q17453085
Q19754082
Q3852590
Q3770555
Q1757617
Q1555828
Q470377
Q3339580
Q19753788
Q2019868
Q1372695
Q19754125
Q19997569
Q481474
Q972400
Q1361667
Q19997557
Q3694262
Q1160847
Q3769699
Q19753755
Q4007892
Q323009
Q2262449
Q3438258
Q962495
Q183433
Q16271446
Q19753888
Q176938
Q981971
Q3107317
Q2474041
Q3694305
Q776990
Q7002138
Q6700132
Q3849075
Q3904250
Q15997104
Q59533611
Q2824734
Q2527217
Q177450
Q3849335
Q1089074
Q558155


In [67]:
data.head()



,Full Name,Gender,Collection,Keeper,Historian Entity
0,Maria Accascina,female,NaN,Comune di Palermo\n,Q98804253
1,Leonardo Agostini,male,NaN,Scuola Normale Superiore,Q1054161
2,Vittorio Alfieri,male,NaN,Biblioteca Medicea Laurenziana,
3,Giuseppe Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,
4,Leopoldo Alinari,male,Archivio Alinari,Museo Nazionale Alinari della Fotografia,


In [ ]:
# new query, to test
# just written and tested on wikidata interface, it returns 4 results for vittorio alfieri and the first is wrong
# but for some entities it works!

# the unions are all different ways to say "italy"
new_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?artHistorian

WHERE {{
  
    { ?artHistorian wdt:P27 wd:Q172579 } UNION 
    { ?artHistorian wdt:P27 wd:Q38 } UNION 
    { ?artHistorian wdt:P27 wd:Q223936 } UNION
    { ?artHistorian wdt:P27 wd:Q48742118 } UNION
    { ?artHistorian wdt:P27 wd:Q5343710 } .
  
    OPTIONAL {{
      ?artHistorian wdt:P1412 wd:Q652
    }}
  
    ?artHistorian rdfs:label ?o 
                  FILTER ( str(?o) = "Giuseppe Alinari" )  .
    
  
    ?artHistorian wdt:P31 wd:Q5 .
}}

"""

